In [3]:
import numpy as np
import pandas as pd
import random
import math
import csv

# def PW_mechanism(t, epsilon):
#   if t<-1.0:
#     t=-1.0
#   elif t>1.0:
#     t=1.0
#   else:
#     pass
#   # print("t:", t)
#   C = (((math.e)**(epsilon/2)+1)/((math.e)**(epsilon/2)-1))
#   # p = ((math.e)**epsilon-(math.e)**(epsilon/2))/(2*math.e**(epsilon/2)+2)
#   l = (C+1)*t/2-(C-1)/2
#   r = l+C-1
#   x = random.uniform(0, 1)
#   if x < (math.e)**(epsilon/2)/((math.e)**(epsilon/2)+1):
#     return random.uniform(l,r)
#   else:
#     choice_prob = [(t+1)/2, (-t+1)/2]
#     chosen_area = np.random.choice([0,1], p=choice_prob)
#     if chosen_area==0:
#       return random.uniform(-C, l)
#     else:
#       return random.uniform(r, C)

# def multiple_PW(t, epsilon):
#   C = (((math.e)**(epsilon/2)+1)/((math.e)**(epsilon/2)-1))
#   d = len(t)
#   ret = [0 for i in range(d)]
#   k = max(1, min(d, int(epsilon/2.5)))
#   tmp_lst = [i for i in range(d)]
#   idx_lst = np.random.choice(tmp_lst, k, replace=False)
#   # print(idx_lst)
  
#   for j in tmp_lst:
#     if j in idx_lst:
#       x = PW_mechanism(t[j],epsilon/k)
#       ret[j] = d*x/k
#     else:
#       ret[j] = random.uniform(-C,C)
  
#   # for j in idx_lst:
#   #   x = PW_mechanism(t[j],epsilon/k)
#   #   ret[j] = d*x/k

#   return ret

# def noising_purpose_variable(label, epsilon):
#   prob0 = [math.e**epsilon/(1+math.e**epsilon), 1/(1+math.e**epsilon)]
#   prob1 = [1/(1+math.e**epsilon), math.e**epsilon/(1+math.e**epsilon)]
#   if label==0.0:
#     label = np.random.choice([0,1], p=prob0)
#   else:
#     label = np.random.choice([0,1], p=prob1)
  
#   return label

# cnt = 0
# for i in range(1000):
#   if noising_purpose_variable(0.0, 1.0)==0.0:
#     cnt+=1
# print(cnt)


In [ ]:
# def multiple_PW_columns_k(t, epsilon, k):
#   d = len(t)
#   ret = [0 for i in range(d)]
#   tmp_lst = [i for i in range(d)]
#   idx_lst = np.random.choice(tmp_lst, k, replace=False)
#   # print(idx_lst)
  
#   for j in idx_lst:
#     x = PW_mechanism(t[j],epsilon/k)
#     ret[j] = d*x/k
  
#   return ret

In [8]:
from sklearn import svm
from sklearn.datasets import load_iris
import joblib
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem
import pandas as pd
import numpy as np
import random
import math
import csv
import sys
import copy

def learning_open_test(train_data, test_data, clf):
  # 説明変数：
  train_X = train_data.iloc[:, 1:].values
  # 目的変数：
  train_y = train_data.iloc[:,0].values

  # 学習（SVM）
  # clf = svm.SVC(gamma=0.5, C=1., kernel='rbf')
  # clf = svm.SVC(kernel='rbf', random_state=None)
  clf.fit(train_X, train_y)
  # # 学習結果を出力

  # 説明変数:
  test_X = test_data.iloc[:, 1:].values
  # 目的変数：
  test_y = test_data.iloc[:,0].values

  # 学習結果の検証（テスト用データx1, x2を入力）
  predicted_y = clf.predict(test_X)
  # 正解データと予測データを比較し、スコアを計算
  score = metrics.accuracy_score(test_y, predicted_y)


  # print("Score:", score)
  return score

def learning_closed_test(train_data, clf):
  # 説明変数：
  train_X = train_data.iloc[:, 1:].values
  # 目的変数：
  train_y = train_data.iloc[:,0].values

  # 学習（SVM）
  # clf = svm.SVC(gamma=0.5, C=1., kernel='rbf')
  # clf = svm.SVC(kernel='rbf', random_state=None)
  clf.fit(train_X, train_y)
  # # 学習結果を出力

  # 説明変数:
  test_X = train_data.iloc[:, 1:].values
  # 目的変数：
  test_y = train_data.iloc[:,0].values

  # 学習結果の検証（テスト用データx1, x2を入力）
  predicted_y = clf.predict(test_X)
  # 正解データと予測データを比較し、スコアを計算
  score = metrics.accuracy_score(test_y, predicted_y)


  # print("Score:", score)
  return score

def evaluate_cross_validation(clf, x, y, K):
  cv = KFold(K, shuffle=True,random_state=0)
  scores = cross_val_score(clf,x,y,cv=cv)
  # print(scores)
  # print ("Mean score: {} (+/-{})".format( np.mean (scores), sem(scores)))

  return np.mean(scores)

def get_split_data(data, num):
  split_idxs = np.random.choice([i for i in range(len(data))], num, replace=False)
  split_data = data.iloc[split_idxs,:]
  # print(split_data)
  return split_data

def main():
  # clf = svm.SVC(gamma=0.5, C=1., kernel='rbf')
  clf = svm.SVC(kernel='rbf', random_state=None)
  filename = "wdbc_reg.csv"
  raw_data = pd.read_csv(filename, header=None)                                        #主キー（不要情報）つき
  data = raw_data.iloc[:,2:]                                              #目的変数が2重
  data.iloc[:,0] = data.iloc[:,0].astype('int')
  # print(data)
  # train_data = data.iloc[0:528,1:]
  # test_data = data.iloc[528:,1:]

  # closed_test
  closed_test_score = learning_closed_test(data, clf)
  # open_test(一回だけ)
  open_text_score = learning_open_test(data.iloc[:len(data)//2,:], data.iloc[len(data)//2:,:], clf)
  # open_test(交差検定用)
  true_score = evaluate_cross_validation(clf, x=data.iloc[:,1:].values, y=data.iloc[:,0].values, K=10)

  print("closed_test_score:", closed_test_score)
  print("open_text_score:", open_text_score)
  print("cross_test_score:", true_score)

  data_num_list = [20, 100, 300, len(data)]
  sim_time = 100
  result = []
  for data_num in data_num_list:
    ctest_scores, otest_scores, cross_scores = [], [], []
    for time in range(sim_time):
      copy_data = copy.deepcopy(data)
      split_data = get_split_data(copy_data, data_num)
      ctest_scores.append(learning_closed_test(split_data, clf))
      otest_scores.append(learning_open_test(split_data.iloc[:len(split_data)//2,:], split_data.iloc[len(split_data)//2:,:], clf))
      cross_scores.append(evaluate_cross_validation(clf, x=split_data.iloc[:,1:].values, y=split_data.iloc[:,0].values, K=10))
    print("data_num:", data_num)
    print("closed_test_scores:", np.mean(ctest_scores))
    print("open_test_scores:", np.mean(otest_scores))
    print("cross_test_scores:", np.mean(cross_scores))
    print()
    result_one = [data_num, np.mean(ctest_scores), np.mean(otest_scores), np.mean(cross_scores)]
    result.append(result_one)
  result = pd.DataFrame(result, columns=["data_num", "closed_test", "open_test", "cross_test"])
  print(result)



  # fig1 = plt.figure()
  # ax1 = fig1.add_subplot(111)
  # ax12 = ax1.twinx()
  # # ax12 = fig1.add_subplot(212)
  # fig2 = plt.figure()
  # ax2 = fig2.add_subplot(111)

  # ax1.set_title("accuracy and number of noised attributes")
  # ax1.set_xlabel("epsilon")
  # ax1.set_ylabel("accuracy")
  # # ax1.plot(epsilon_lst, true_score_lst, color="red", label="non-noise")
  # # ax1.plot(epsilon_lst, mean_score_lst, color="blue", label="noised")
  # ax1.set_ylim(0.5,1.0)
  # ax12.set_ylim(0,34)
  # # ax12.plot(epsilon_lst, k_lst, linewidth = 2, label="k_num",linestyle="dashed")
  # ax12.set_ylabel("k")
  # h1, l1 = ax1.get_legend_handles_labels()
  # h2, l2 = ax12.get_legend_handles_labels()
  # ax1.legend(h1+h2, l1+l2, loc='lower right')

  # # ax2.set_title("scores")
  # # ax2.set_xlabel("epsilon")
  # # ax2.set_ylabel("score")
  # # ax2.plot(epsilon_lst, mean_score_lst, color="blue", label="noised")
  # # ax2.plot(epsilon_lst, reduced_mean_score_lst, color="green", label="noised_and_reduced")
  # # ax2.legend()
  # # plt.show()
  # ax2.set_title("accuracy with fixed k")
  # ax2.set_xlabel("epsilon")
  # ax2.set_ylabel("score")
  # # ax2.plot(epsilon_lst, mean_score_lst, color="blue", label="noised")
  # # ax2.plot(epsilon_lst, mean15_score_lst, color="green", label="noised(k=15)")
  # # ax2.plot(epsilon_lst, mean20_score_lst, color="m", label="noised(k=20)")
  # # ax2.plot(epsilon_lst, mean25_score_lst, color="black", label="noised(k=25)")
  # ax2.legend()
  # plt.show()
  # # plt.title("scores")
  # # plt.xlabel("epsilon")
  # # plt.ylabel("score")
  # # plt.plot(epsilon_lst, true_score_lst, color="red", label="non-noise")
  # # plt.plot(epsilon_lst, mean_score_lst, color="blue", label="noised")
  # # plt.plot(epsilon_lst, reduced_mean_score_lst, color="green", label="noised_and_reduced")
  # # plt.legend()

main()

C:\Users\laplacian\AppData\Local\Temp\ipykernel_63148\3885700970.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:,0] = data.iloc[:,0].astype('int')


closed_test_score: 0.9806678383128296
open_text_score: 0.9719298245614035
cross_test_score: 0.9771929824561404


ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:

def read_data(filename):
  # filename = "wdbc_reg.csv"
  raw_data = pd.read_csv(filename)
  return raw_data

def normalize(data):
  for column_name, column in data.iteritems():
    

In [1]:
import statistics, time
import multiprocessing as mp
from pathlib import Path
from sklearn import svm, metrics
from dataset.wdbc_dataset import WDBC_Parser
from dataset.methods import *
#This is the program to drow the PW noose to all attributions

to_mean = lambda l: f"{round(statistics.mean(l)*100, 2)}%"

def gen_data(_eps=-1, seed=0xE39FE39F, rand_label=False):
    dataset = WDBC_Parser("./dataset")

    if _eps != -1:
        eps = _eps / 34
        dataset.PW(eps, seed)
        if rand_label:
            dataset.random_label(eps, seed)
    
    return dataset

def all_raw():
    accs, recalls, precisions = [], [], []
    train_set = gen_data()
    test_set = gen_data()

    for i in range(10):
        data_train, label_train = train_set.split(i)
        clf = svm.SVC(C=2.1)
        clf.fit(data_train, label_train)

        data_test, label_test = test_set.split(i, True)
        pre = clf.predict(data_test)
        ac_score = metrics.accuracy_score(label_test, pre)
        recall = metrics.recall_score(label_test, pre)
        precision = metrics.precision_score(label_test, pre, zero_division=0)
        accs.append(ac_score)
        recalls.append(recall)
        precisions.append(precision)
    
    with open(f"result_wdbc/primitive.csv", 'a') as ofile:
        ofile.write(f"-1,{to_mean(accs)},{to_mean(recalls)},{to_mean(precisions)}\n")

def ten_seed(eps=3):
    accs, recalls, precisions = [], [], []

    seeds = [pow(0xE39FE39F, i, 1 << 32) for i in range(1, 11)]
    for seed in seeds:
        train_set = gen_data(eps, seed, True)
        test_set = gen_data(eps, seed)

        for i in range(10):
            data_train, label_train = train_set.split(i)
            clf = svm.SVC(C=2.1)
            clf.fit(data_train, label_train)

            data_test, label_test = test_set.split(i, True)
            pre = clf.predict(data_test)
            ac_score = metrics.accuracy_score(label_test, pre)
            recall = metrics.recall_score(label_test, pre)
            precision = metrics.precision_score(label_test, pre, zero_division=0)
            accs.append(ac_score)
            recalls.append(recall)
            precisions.append(precision)
    
    return to_mean(accs), to_mean(recalls), to_mean(precisions)

def wait_single(procs):
    try:
        procs[0].join()
    except KeyboardInterrupt:
        for p in procs:
            p.terminate()
        exit()

def wait_all(procs):
    try:
        for p in procs:
            p.join()
    except KeyboardInterrupt:
        for p in procs:
            p.terminate()
        exit()

def one_proc(begin, size):
    eps = begin
    while eps < begin + size +0.01:
    #for eps in range(begin, begin+size*5, 5):
        got = ten_seed(eps)
        Path(f"result_wdbc").mkdir(parents=True, exist_ok=True)
        with open(f"result_wdbc/primitive.csv", 'a') as ofile:
            ofile.write(f"{eps},{got[0]},{got[1]},{got[2]}\n")
        eps +=0.2
        eps = round(eps,2)

def all_batch(max_thread=20):
    procs = []
    for eps in range(5, 50, 15):
        p = mp.Process(target=one_proc, args=(eps, 15))
        procs.append(p)
        p.start()

        if len(procs) == max_thread:
            wait_single(procs)
            procs = procs[1:]
    
    wait_all(procs)

if __name__ == '__main__':
    print("Begin:", time.asctime())
    start = time.time()
    all_batch()
    all_raw()#ノイズを載せない場合の値
    print(f"Time usage: {time.time() - start}")


ModuleNotFoundError: ignored